## Application

This notebook contains everything needed to run the routing application for our project. The notebook contains four main parts:
1. Data loading and initalization of the network
    - Here we take care of the necessities. More details about our data pipline can be found in data_pipeline.ipynb
2. Delay modelling
    - This section shows the final predictive model of delays used in the project. Feature engineering and other modelling efforst is found in predictive_modelling.ipynb
3. Routing algorithm
    - This section contains the final routing algorithm used.
4. Testing and visualization
    - Testing and visualization is the final product of the notebook. Here, one can interact with a UI to the route planner to test the application.
    
To be able to run the testing of our application you should run all cells in the notebook. In the bottom under the title 'Application', you find the testing interface. Be aware, some queries may take up to 1.5 minutes, but most take around 20 seconds. 

If something is not working, please contact us at:
benjamin.rike@epfl.ch or Slack

__Group Gutane:__

Andreas Aarrestad\
Benjamin Rike\
Haakon Døssland\
Olav Førland

## Initialization

In [ ]:
%%local 

import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
from IPython import get_ipython
from pyhive import hive
import networkx as nx
import numpy as np

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.set_option("display.max_columns", 50) 
pd.options.mode.chained_assignment = None


username = os.environ['RENKU_USERNAME']
hiveaddr = os.environ['HIVE_SERVER2']
(hivehost,hiveport) = hiveaddr.split(':')
print("Operating as: {0}".format(username))

## Model of public transport infrastructure

We start by loading the relevant data from HDFS by using Hive

In [ ]:
%load_ext sparkmagic.magics
# If using Python 3 kernel

In [ ]:
%%local 
# Set up spark session
server = 'http://iccluster029.iccluster.epfl.ch:8998'

packages = """{"packages": "graphframes:graphframes:0.6.0-spark2.3-s_2.11"}"""

# Set application name as "<your_gaspar_id>-final_project"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell=f"""{{ "name": "{username}-final_project", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G", "conf": {packages}}}"""
)
# Send username to spark channel
get_ipython().run_line_magic(
    "spark", "add -s {0}-final_project -l python -u {1} -k".format(username, server)
)

In [ ]:
print('We are using Spark %s' % spark.version)

In [ ]:
%%spark
import pyspark.sql.functions as F

In [ ]:
%%local 
# create connection
conn = hive.connect(host=hivehost, 
                    port=hiveport,
                    username=username) 
# create cursor
cur = conn.cursor()

In [ ]:
%%local
# Retrieve stops to local

query = """
DROP TABLE IF EXISTS {0}.stops
""".format(username)
cur.execute(query)

query = """
    CREATE EXTERNAL TABLE {0}.stops(
        stop_id string,
        stop_name string,
        stop_lat double,
        stop_lon double,
        distance double
    )
    STORED AS orc
    LOCATION '/group/gutane/stops'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.stops 
""".format(username)
stops = pd.read_sql(query, conn)
stops = stops.rename(mapper=lambda x: x.split('.')[1], axis=1)

In [ ]:
%%local
# Retrive walking connections to local

query = """
    DROP TABLE IF EXISTS {0}.walking_connections
""".format(username)
cur.execute(query)

query = """
    CREATE EXTERNAL TABLE {0}.walking_connections(
        source_id string, 
        destination_id string, 
        travel_time int
    )
    STORED AS ORC
    LOCATION '/group/gutane/walking_connections'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.walking_connections 
""".format(username)
walking_connections = pd.read_sql(query, conn)
# Fix column names
walking_connections = walking_connections.rename(mapper=lambda x: x.split('.')[1], axis=1)

In [ ]:
%%local 
# Retrieve connections to local

query = """
    DROP TABLE IF EXISTS {0}.connections
""".format(username)
cur.execute(query)

query = """
    CREATE EXTERNAL TABLE {0}.connections(
        trip_id string,
        src_id string, 
        src_arrival_time string,
        src_departure_time string, 
        dest_id string,
        dest_arrival_time string, 
        dest_departure_time string, 
        stop_sequence int, 
        travel_time_from_prev int,
        cummulated_travel_time int, 
        route_short_name string,
        route_desc string 
    )
    STORED AS ORC
    LOCATION '/group/gutane/connections'
""".format(username)
cur.execute(query)

query = """
    select * from {0}.connections 
""".format(username)
connections = pd.read_sql(query, conn)
connections = connections.rename(mapper=lambda x: x.split('.')[1], axis=1)

## Network modelling

At first, we create a network with stops as nodes and connections and walking connections as edges. The graph is directed as a transport only drive one way. We then inspect whether the graph is strongly connected. 

In [ ]:
%%local
# defining edges and nodes
edges = connections[['src_id', 'dest_id']].to_numpy()
edges = edges[edges[:, 1] != None] # Remove edges from end stops
walking_edges = walking_connections[['source_id', 'destination_id']].to_numpy()
edges = np.vstack([edges, walking_edges])

nodes = stops['stop_id']

# Creating directed graph
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# checking number of strongly connected components
num_connected = nx.number_strongly_connected_components(G)
comp = nx.strongly_connected_components(G)
largest_comp = max(comp, key=len)
percentage_lcc = len(largest_comp) / G.number_of_nodes() * 100

print('The largest component has', len(largest_comp), 
      'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')

We filter away stops not in the largest component, as mentioned as a valid approach during the Q&A lecture. This is because the only way to come to these stops is to go through stops outside the 15km radius. 

In [ ]:
%%local
# finding the largest component and creating a new subgraph from this
Gcc = sorted(nx.strongly_connected_components(G), key=len, reverse=True)
G2 =  G.subgraph(Gcc[0])

# only keeping connections in G2
temp = connections[['src_id', 'dest_id']].to_numpy()
connections['keep'] = [G2.has_edge(x[0], x[1]) for x in temp]
connections = connections[connections.keep==True]
connections.drop('keep', axis=1, inplace=True)

# only keeping walking connections in G2
temp = walking_connections[['source_id', 'destination_id']].to_numpy()
walking_connections['keep'] = [G2.has_edge(x[0], x[1]) for x in temp]
walking_connections = walking_connections[walking_connections.keep==True]
walking_connections.drop('keep', axis=1, inplace=True)

# only keeping stops in G2
stops['keep'] = stops['stop_id'].apply(lambda x: G2.has_node(x))
stops = stops[stops.keep == True]
stops.drop('keep', axis=1, inplace=True)

In [ ]:
%%local

# repating the former calculation and checking whether the whole graph is connected now
edges = connections[['src_id', 'dest_id']].to_numpy()
edges = edges[edges[:, 1] != None] # Remove edges from end stops
walking_edges = walking_connections[['source_id', 'destination_id']].to_numpy()
edges = np.vstack([edges, walking_edges])

nodes = stops['stop_id']

# Create graph
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

num_connected = nx.number_strongly_connected_components(G)
comp = nx.strongly_connected_components(G)
largest_comp = max(comp, key=len)
percentage_lcc = len(largest_comp) / G.number_of_nodes() * 100

print('The largest component has', len(largest_comp), 
      'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')

We have achieved a fully strongly connected graph.

### Analysis of delays



As we decided to implement the routing algorithm locally, the inference from the predictive model must also be local. We decided to drop the ml predictive model of two reasons. 1) The model we developed was not much better than baseline. There was almost no correlation between our features and the delay. The only thing that correlated strongly was the departure at the last stop. However, we do not have access to this in real-time and can therefore not use it as a feature. If we were to make our work production-ready, we would accquire this data such that we could use real-time delays to predict future delays. 2) it was dificult to figure out how to move ml weights out of Spark and then run the model. Therefore, we rather decided to fit a probability distribution to the delays and use this as a measure of uncertainty.

More information about our approaches to predictive modelling can be found in predictive_modelling.ipynb, but we present the benfits and drawbacks in short here:

__Benfits__
- Easy and simple
- Work directly with a probability distribution which is what we try to model

__Drawbacks__
- We don't use the predictive power of the other features that we know, so it is not as accuracte as it could be

If we have more time, we would use more time creating a better predictive model. It could also be interesting to incoporate real-time data into the modelling as the strongest correlator of delay is previous delay, but this is a large scope.


In delays, there are two things we want to model: 1) How likely you are to miss a connections, and 2) How likely you are to not arrive at time at the last stop.

Define the following variables:

$T_D$ = Actual time of departure\
$T_A$ = Actual time of arrival\
$S_D$ = Scheduled time of departure\
$S_A$ = Scheduled time of arrival\
$A$ = Delay in arrival time\
$D$ = Delay in departure time\

We then want to find for 1):

$P({T_D-T_A \geq 0 })$
= $P({S_D + D - S_A + A \geq 0})$
= $P({A-D \leq S_D - S_A})$

By also accounting for transfer time are going to model:\
$P({A-D \leq S_D - S_A-2})$

which corresponds to the CDF of the random variable $A-D$. We must therefore find this distribution.

In 2), we only need to model:

$P({A \leq B_T - S_A})$

where $B_T$ = the arrival by time.



In [ ]:
# reading data from hdfs
sample = spark.read.orc('/group/gutane/processed_features')

In [ ]:
sample.printSchema()

In [ ]:
sample.count()

In [ ]:
#in the predictive modelling notebook we have seen that transport type is what correlates the most with delay
# and that train have different delays than bus and tram
# therefore, we split in train and other for the model fittings

# creating new column for departure delay
sample = sample.withColumn('departure_delay', F.col('actual_departure_time').cast('long')-F.col('scheduled_departure_time').cast('long'))
sample = sample.filter(sample.departure_delay.isNotNull())

#taking a sample of departure and arrival delays
delay_train = sample.filter(sample.transport_type=='Zug').select('delay', 'departure_delay', 'transport_type')
delay_other = sample.filter(sample.transport_type!='Zug').select('delay', 'departure_delay', 'transport_type')
delay_train = delay_train.sample(0.1, seed=41)
delay_other = delay_other.sample(0.1, seed=41)


In [ ]:
%%spark -o delay_train -t df -n 5000

In [ ]:
%%spark -o delay_other -t df -n 5000

First, we model the arrival delays

In [ ]:
%%local
import seaborn as sns
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay, ax=ax[0])
sns.histplot(delay_other.delay, ax=ax[1])
ax[0].set_xlim(-500,1000)
ax[0].set_title("Delay distribution for trains")
ax[1].set_xlim(-500,1000)
ax[1].set_title("Delay distribution for other transport types")

We observe that delay follows something that looks like a right-skewed normal distribution. The right tail is longer than the left. We therefore, try estimating the delays with a skewed normal distribution. It does also look like a log-normal distribution, but since it takes on negative values, and there is no left bound, this is not the case.

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm

# fitting the skewed normal distribution
a_t, loc_t, scale_t = stats.skewnorm.fit(delay_train.delay)
a_o, loc_o, scale_o = stats.skewnorm.fit(delay_other.delay)

print("Estimated values")
print("a                  loc                scale")
print(a_t, loc_t, scale_t)
print(a_o, loc_o, scale_o)

In [ ]:
%%local
import numpy as np
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay, ax=ax[0], stat='density', label = "empirical density")
sns.histplot(delay_other.delay, ax=ax[1], stat='density', label = "empirical density")

x1 = np.linspace(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t), 100)
x2 = np.linspace(skewnorm.ppf(0.00001, a_o, loc=loc_o, scale=scale_o),
                skewnorm.ppf(0.99999, a_o, loc=loc_o, scale=scale_o), 100)

ax[0].plot(x1, skewnorm.pdf(x1, a_t, loc=loc_t, scale=scale_t),
       'r-', lw=5, alpha=0.6, label='fitted pdf')
ax[1].plot(x2, skewnorm.pdf(x2, a_o, loc=loc_o, scale=scale_o),
       'r-', lw=5, alpha=0.6, label='fitted pdf')

ax[0].set_xlim(-500,1000)
ax[1].set_xlim(-500,1000)

ax[0].set_title("Distribution of delay and the fitted skewed normal pdf for trains")
ax[1].set_title("Distribution of delay and the fitted skewed normal pdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

In [ ]:
%%local
import numpy as np

fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.ecdfplot(delay_train.delay, ax=ax[0], lw=2, label = "ECDF")
sns.ecdfplot(delay_other.delay, ax=ax[1], lw=2, label = "ECDF")


x1 = np.linspace(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t), 100)
x2 = np.linspace(skewnorm.ppf(0.00001, a_o, loc=loc_o, scale=scale_o),
                skewnorm.ppf(0.99999, a_o, loc=loc_o, scale=scale_o), 100)

ax[0].plot(x1, skewnorm.cdf(x1, a_t, loc=loc_t, scale=scale_t),
       'r-', lw=2, alpha=1, label='fitted CDF')
ax[1].plot(x2, skewnorm.cdf(x2, a_o, loc=loc_o, scale=scale_o),
       'r-', lw=2, alpha=1, label='fitted CDF')

ax[0].set_xlim(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t))
ax[1].set_xlim(skewnorm.ppf(0.00001, a_t, loc=loc_t, scale=scale_t),
                skewnorm.ppf(0.99999, a_t, loc=loc_t, scale=scale_t))

ax[0].set_title("Empirical cumulative distribution of delay and the fitted skewed normal cdf for trains")
ax[1].set_title("Empirical cumulative distribution of delay and the fitted skewed normal cdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

We see that the distribution fit the data quite good. It does quite capture the spike around the mean, but otherwise good. Especially, it seems to capture the tails which may be the most important for our task as large delays are more capable of destroying a route plan. However, a weakness in our model is that we will underestimate the number of delays between 50 and 150 seconds for other transportation. Here, there is room for improvement.

In [ ]:
%%local
print("Probability that the delay is smaller than 60 seconds for trains: %.3f"%stats.skewnorm.cdf(60,a_t, loc_t, scale_t))
print("Probability that the delay is smaller than 60 seconds for other transport types: %.3f"%stats.skewnorm.cdf(60,a_o, loc_o, scale_o))


Then, we model the difference in arrival and departure delay.

In [ ]:
%%local
fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay-delay_train.departure_delay, ax=ax[0])
sns.histplot(delay_other.delay-delay_other.departure_delay, ax=ax[1], bins=100)
ax[0].set_xlim(-200,200)
ax[0].set_title("Arrival minus departure delay distribution for trains")
ax[1].set_xlim(-100,100)
ax[1].set_title("Arrival minus departure delay distribution for other transport types")

The arrival minus distributions are harder to fit. The above one looks like some kind of Laplace distribution or similar. We tried several modelling approaches below, and found that the Laplace fits best. For the other transport types, it is hard to find a fitting distributions, but we also here fit a Laplace distribution as we assume it should be similar as for trains. This however looks like a bad assumption and something we should attempt fixing, but we unfortunatley ran out of time.

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm, cauchy

# fitting the skewed normal distribution
a_t_stops, loc_t_stops, scale_t_stops = stats.skewnorm.fit(delay_train.delay-delay_train.departure_delay)
a_o_stops, loc_o_stops, scale_o_stops = stats.skewnorm.fit(delay_other.delay-delay_other.departure_delay)

print("Estimated values")
print("a                  loc                scale")
print(a_t, loc_t, scale_t)
print(a_o, loc_o, scale_o)

In [ ]:
%%local
from scipy import stats
from scipy.stats import skewnorm, cauchy, norm, laplace

# fitting the skewed normal distribution
loc_train_laplace, scale_train_laplace = stats.laplace.fit(delay_train.delay-delay_train.departure_delay)
loc_other_laplace, scale_other_laplace = stats.laplace.fit(delay_other.delay-delay_other.departure_delay)

fig, ax = plt.subplots(2,1,figsize=(12,12))
sns.histplot(delay_train.delay-delay_train.departure_delay, ax=ax[0], stat='density', label = "empirical density")
sns.histplot(delay_other.delay-delay_other.departure_delay, ax=ax[1], stat='density', label = "empirical density", bins=50)

x1 = np.linspace(laplace.ppf(0.00001, loc_train_laplace, scale_train_laplace),
                laplace.ppf(0.99999,loc_train_laplace, scale_train_laplace), 100)
x2 = np.linspace(laplace.ppf(0.00001, loc_other_laplace, scale_other_laplace),
                laplace.ppf(0.99999,  loc_other_laplace, scale_other_laplace), 100)

ax[0].plot(x1, laplace.pdf(x1, loc_train_laplace, scale_train_laplace),
       'r-', lw=5, alpha=0.6, label='fitted pdf')
ax[1].plot(x2, laplace.pdf(x2,  loc_other_laplace, scale_other_laplace),
       'r-', lw=5, alpha=0.6, label='fitted pdf')

ax[0].set_xlim(-200,200)
ax[1].set_xlim(-200,200)

ax[0].set_title("Distribution of delay differences and the fitted laplace pdf for trains")
ax[1].set_title("Distribution of delay differences and the fitted laplace pdf for other transportation")
ax[0].set_xlabel("Delay")
ax[0].set_ylabel("Density")
ax[1].set_xlabel("Delay")
ax[1].set_ylabel("Density")
ax[0].legend()
ax[1].legend()

The Laplace distribution does an ok job of fitting the difference for trains, but not very much for other transportation. This is a weakness of our model, and something we would like to work more on, but we ran out of time and prioritize improving the routing algorithm. Further work should try developing a model that uses more of the available information to model the distributions.

## Route planning algorithm

#### Network 
As the routing requires a customizable network, we decided to code it ourselves from scratch. Each node in the network represents a stop and each edge is a time dependant direct route between two stops. This is implemented by storing a table of all routes in each node to its adjacent nodes indexed by the adjacent node itself. This ensures fast lookup during search, but makes for a larger overhead during graph creation. 

#### Search 
When the graph has been built, there's a range of different flags and values that are set throughout the search. These dynamic variables are initialized with default values during each run, while the static variables remain constant. 

#### Optimality
As each stop has a physical location, we decided to use a customized version of the best-first search algorithm A*. It is an extension of Dijkstra's algorithm where the functionality for picking the next node to traverse is also based on an estimate of the distance to the destination. This heuristic function is in our case the euclidean distance, which will always be less or equal to the actual shortest path to the destination. As such, the heuristic is admissable and will therefore ensure optimality. 

The choosing of the next node to traverse is implemented by the use of a minheap with an index given by the sum of the elapsed time and the distance heuristic, while the corresponding value is a reference to the node.

#### Time variance
As the edge cost in our case between two nodes is not static, and based on the elapsed time to the node, we have to modify the algorithm to do a lookup of the edge cost during each new route evaluation. This makes our algorithm a bit slower, but is not substantial because of the table distribution during graph population.

#### Uncertainty
When we are modelling the fastest route at a certain confidence level, the delay distribution is essential. To simplify task, we have assumed that there is no correlation between delays. (a simplification that we have seen in predictive_modelling.ipynb to be very wrong) . We calculate uncertainties every time the user is going to change transport. That means that we calcualte the probability that he is able to reach the next departure. This is modelled by calculating the CDF-value for the difference in scheduled arrival time and scheduled departure time.

We also calculate the uncertainty at the end stop for the arrival time. Here we also assume no correlation between delays such that this is just the cdf of the difference of arrival_by_time and scheduled_arrival_time. We then cumulativley multiply the probabilities for each route and discard a route if the probability is lower than the maximum 

#### Other heuristics and assumptions
- To disincentivize changing transportation too often, a two minute transfer time is added.  
- To choose to walk a distance, the walk must be 2 minutes faster than taking a transport. We chose this as it fits with what we would wish in real life. It is worth waiting 2 minutes if we otherwise must walk
- We don't account for switching route in the case of delay. Once a route is planned, we expect the user to follow it.
- The algorithm do not take into account the consequences of missing one connection vs. another

In [ ]:
%%local
import heapq
import numpy as np

In [ ]:
%%local
# removing nans and some formatting
route_connections = connections[connections['dest_id'].notna()]
for i in ['src_arrival_time','src_departure_time','dest_arrival_time','dest_departure_time']:
    route_connections.loc[i,:] = route_connections[i].apply(lambda d : pd.Timestamp(d))
    
# smaller sample while testing    
route_connections = route_connections#.head(3000)

In [ ]:
%%local 
labels = ['trip_id', 'src_id', 'src_arrival_time', 'src_departure_time', 
          'dest_id', 'dest_arrival_time', 'dest_departure_time', 'stop_sequence', 
          'travel_time_from_prev', 'cummulated_travel_time', 'route_short_name', 
          'route_desc', 'cum_confidence', 'arrival_confidence']

# Takes a column name and returns the corresponding index, in the 
# order the columns are in the dataframes
def get_index(column):
    label_map = \
        {c:i for i, c in enumerate(labels)}
    return label_map[column]


In [ ]:
%%local
class Node:
    def __init__(self, stop_id, stop_name, lat, lon):
        # STATIC PARAMETERS
        self.stop_id = stop_id
        self.stop_name = stop_name
        self.lat = lat
        self.lon = lon
        self.routes = {} # destination_stop_id -> table[tripID, departure_timestamp, arrival_timestamp, route_short_name]
        self.walking_connections = {} # destination_stop_id -> time
         
        # DYNAMIC PARAMETERS
        self.intialize_dynamic_parameters()
        
    def intialize_dynamic_parameters(self):
        self.parent = None
        self.origin_route = None
        self.has_gained_optimum = False
        self.is_under_consideration = False
        self.elapsed_time = float('inf')
    
    def __lt__(self, other):
        return self.stop_id < other.stop_id
    
    def __le__(self, other):
        return self.stop_id <= other.stop_id
    
    def add_route_timetable(self, adjacent_node_id, timetable):
        self.routes[adjacent_node_id] = timetable 
    
    def add_walking_connection(self, adjacent_node_id, walking_time):
        self.walking_connections[adjacent_node_id] = walking_time
    
    def calculate_confidence(self, trips):
        """Calculating the confidence for being in time for the departure time at the next stop.
        Currently, only accounts for the delay in arrival time, not in departure time. This leads to too conservative estimates."""
        
        #TODO: Add suport for departure modelling too?
        # If so, we need to change docstring too.
        # extracting the transport vehicle of every trip
        transport = trips[:, get_index('route_desc')]
        
        # assinging 1 in confidence if there is no previous route
        # this is because we assume one is not too late to the first departure
        if self.origin_route == None:
            confidence = np.asarray([1 for element in transport])
            return confidence
        
        
        if isinstance(self.origin_route, np.ndarray):
            # calculating the time between scheduled arrival and departure
            arrival_time = self.origin_route[-1].get('dest_arrival_time')
            if isinstance(arival_time, str):
                diff_time = (trips[:,get_index('src_departure_time')] -
                         pd.Timestamp(arrival_time).to_pydatetime())
            else:
                diff_time = (trips[:,get_index('src_departure_time')] -
                         (arrival_time).to_pydatetime())
                
            # accounting for transfer time
            diff_time = diff_time - pd.Timedelta(2)
            
            # calculating the confidence by using the estimated cdf
            f = lambda x: x.seconds
            vf = np.vectorize(f)
            confidence = np.where(transport=="Zug",
                                  stats.laplace.cdf(vf(diff_time), loc_train_laplace, scale_train_laplace),
                                  stats.laplace.cdf(vf(diff_time), loc_train_laplace, scale_train_laplace)
                                 )

            # only using the confidence if one switches route
            # this is because one cannot be too late for as bus one already is on
            confidence = np.where(trips[:, get_index('route_short_name')]!=
                                  self.origin_route[-1].get('route_short_name'),
                                  confidence,
                                  1)
        else:
            # same as in if, just that origin_route isn't a list
            arrival_time = self.origin_route.get('dest_arrival_time')
            if isinstance(arrival_time, str):
                diff_time = (trips[:,get_index('src_departure_time')] -
                         pd.Timestamp(arrival_time).to_pydatetime())
            else:
                diff_time = (trips[:,get_index('src_departure_time')] -
                         (arrival_time).to_pydatetime())
                
            diff_time = diff_time - pd.Timedelta(2)
            
            f = lambda x: x.seconds
            vf = np.vectorize(f)
            
            confidence = np.where(transport=="Zug",
                                  stats.laplace.cdf(vf(diff_time),loc_train_laplace, scale_train_laplace),
                                  stats.laplace.cdf(vf(diff_time), loc_train_laplace, scale_train_laplace)
                                 )
                                 
            confidence = np.where(trips[:, get_index('route_short_name')]!=
                                  self.origin_route.get('route_short_name'),
                                  confidence,
                                  1)
        return confidence
    
    def calc_final_conf(self, trips, by_time):
        """Calculates the confidence with which the trip reaches the destination by the 
        by_time. Only looks on the trip from the next last stop to the last stop
        as we assume i.i.d. delays which do not propogate"""
        transport = trips[:, get_index('route_desc')]
        diff_time = ((trips[:,get_index('dest_arrival_time')]).astype('datetime64[s]') - 
                     np.datetime64(pd.Timestamp(by_time)))
        diff_time = np.asarray([pd.Timedelta(element).seconds for element in diff_time])
        confidence = np.where(transport == "Zug", 
                             stats.skewnorm.cdf((diff_time), a_t, loc_t, scale_t),
                             stats.skewnorm.cdf((diff_time), a_o, loc_o, scale_o))
        return confidence
        
        
    def wait(self, transport):
        # returns the media wait time at a stop while being too late for a given transport type
        return [pd.Timedelta(seconds = 48) if element=="Zug" else pd.Timedelta(12) for element in transport]
    
    def previous_confidence(self):
        # retrieving the previous cumulative confidence by looking up in self.origin_route
        if self.origin_route == None:
            cum_conf_previous = 1
        else:
            if isinstance(self.origin_route, np.ndarray):
                cum_conf_previous = self.origin_route[-1].get('cum_confidence')
            else:
                cum_conf_previous = self.origin_route.get('cum_confidence')
        return cum_conf_previous
    
    def get_optimal_route(self, adjacent_node_id, departure_timestamp, confidence, destination, by_time):
        """Returns the optimal route between two stops"""
        # flag telling if the route is to destination or not
        # will only calculate arrival delay if this is true
        # in the other cases, we only check if the delay makes us miss a connection
        # we do it this way as we assume to correlation betwen delays
        # then, arrival delay is only the delay between the next last and the last stop
        to_final = False
        if destination == adjacent_node_id:
            to_final = True
            
        current_time_timestamp = (pd.Timestamp(departure_timestamp) + pd.Timedelta(seconds=self.elapsed_time)).strftime('%Y-%m-%d %X')
        time = pd.Timestamp(current_time_timestamp)
        transfer_time = pd.Timedelta(minutes=2)
        found_connection = False

        
        # iterating over adjacent nodes
        if adjacent_node_id in self.routes.keys():
            
            
            found_connection = True
            routes_to_adjacent = self.routes[adjacent_node_id]
            
            # converting time strings to datetime
            routes_to_adjacent[:, get_index('src_departure_time')] = routes_to_adjacent[:, get_index('src_departure_time')].astype('datetime64[s]')
            routes_to_adjacent[:, get_index('src_arrival_time')] = routes_to_adjacent[:, get_index('src_arrival_time')].astype('datetime64[s]')
            
            
            if self.origin_route == None:
                # filtering out trips which starts too late
                available_trips = routes_to_adjacent[routes_to_adjacent[:, get_index('src_departure_time')] >= time]
            else:
                # filtering out trips which starts too late compared to time + possible transfer time
                available_trips = (routes_to_adjacent[(routes_to_adjacent[:, get_index('src_departure_time')] >= time+transfer_time)  |
                                                ((routes_to_adjacent[:, get_index('src_departure_time')] >= time) &
                                                 (routes_to_adjacent[:, get_index('route_short_name')] == self.origin_route.get('route_short_name')))]
                              )
            # if there is available trips
            if available_trips.size > 0:
                
                cum_conf_previous = self.previous_confidence()
                # heuristic for speed purposes
                # only calculating confidences for 10 fastest trips
                k = 5
                if available_trips.shape[0]>=k:
                    kth_smallest = np.partition(available_trips[:, get_index('dest_arrival_time')], k-1)[k-1]
                    available_trips = available_trips[available_trips[:, get_index('dest_arrival_time')] <= 
                                                   kth_smallest]
                # for trip in available_trips, calculate confidence that you will reach the departure
                current_confidence = self.calculate_confidence(available_trips) #calculate confidence for every element in available_trips
                #current_confidence = np.asarray([1 for element in range(available_trips.shape[0])])
                # calculating new cumulative confidence as previous times this departure's
                cum_confidence = cum_conf_previous*current_confidence
                # stacking cumulative confidence to available trips
                available_trips = np.hstack([available_trips, np.expand_dims(cum_confidence, 1)])
                # keeping trips which still fullfils the confidence limit


                available_trips = available_trips[available_trips[:, -1] >=
                                                 confidence]
                
                # finally, if the trip is to the destination node
                # we check that we reach it by the necessary arrival time
                if to_final:
                    if available_trips.shape[0]>=k:
                        kth_smallest = np.partition(available_trips[:, get_index('dest_arrival_time')], k-1)[k-1]
                        available_trips = available_trips[available_trips[:, get_index('dest_arrival_time')] <= 
                                                       kth_smallest]
                    final_confidence = self.calc_final_conf(available_trips, by_time)
                    available_trips[:,-1] = available_trips[:,-1]*final_confidence
                    available_trips = available_trips[final_confidence >=
                                                 confidence]
                # checking if there still is available trips
                if available_trips.size > 0:
                    # calculating the trip which reaches the destination the earliest
                    optimal_trip = available_trips[available_trips[:, get_index('dest_arrival_time')] == 
                                                   available_trips[:, get_index('dest_arrival_time')].min()]
                    
                else:
                    found_connection = False
            else:
                found_connection = False
                # calculating the previous cumulative confidence
                cum_conf_previous = self.previous_confidence()
        else:
            cum_conf_previous = self.previous_confidence()
        if not found_connection:
            optimal_trip = np.asarray([[]])
            
        # no connections
        if optimal_trip.size == 0 and (adjacent_node_id not in self.walking_connections.keys()):
            return False
            
        walk = np.inf   
        # checks walking connections
        if adjacent_node_id in self.walking_connections.keys():
            walk = self.walking_connections[adjacent_node_id]
            walk_arrival_time = time + pd.Timedelta(seconds=walk)
            # Pseudo-correct dictionary for walking times -> follows the same format as a trip connection
            walking_trip = {'trip_id': "", 'src_id': self.stop_id, 'src_arrival_time': time, 'src_departure_time': time, 
                      'dest_id': adjacent_node_id, 'dest_arrival_time': walk_arrival_time, 'dest_departure_time': walk_arrival_time,
                      'stop_sequence': 0, 'travel_time_from_prev': 0, 'cummulated_travel_time': 0, 'route_short_name': 'Walk', 
                      'route_desc': 'Walk', 'cum_confidence':cum_conf_previous}
            
            # if no transport trip, we return the walk
            if optimal_trip.size == 0:
                return walking_trip, walk
            
        if optimal_trip.size > 0: 
            trip_arrival_timestamp = optimal_trip[:, get_index('dest_arrival_time')][0]
            elapsed_time = pd.Timedelta(pd.Timestamp(trip_arrival_timestamp) - time).seconds
            
            # if no walking connections, return shortest transport
            if not self.walking_connections:
                res = {l: optimal_trip[-1, :][i] for i, l in enumerate(labels[:optimal_trip.shape[1]])}
                return res, elapsed_time
        
        # if walk is 2 minute faster than transport --> walk
        if walk + 120 < elapsed_time:
            return walking_trip, walk
        
        if True:
            res = {l: optimal_trip[-1, :][i] for i, l in enumerate(labels[:optimal_trip.shape[1]])}
            return res, elapsed_time

    
    def get_all_successor_stop_ids(self):
        if self.routes.keys():
            return self.routes.keys()
        return self.walking_connections.keys()
        
    
    

In [ ]:
%%local
from datetime import datetime
class Network:
    def __init__(self):
        # contains all the stops in the network. edges are stored in the nodes.
        self.nodes = {}

    # add new nodes and their routes to other nodes
    def populate(self, stops, connections, walking_connections):
        """
        Populate each node with its connecting routes 
        @param stops: df with stop_id: string, stop_name: string, stop_lat: double, stop_lon: double
        @param connections: df with trip_id: string, arrival_time: timestamp, departure_time: timestamp, source_id: string, destination_id: string
        """
        for index, stop in stops.iterrows():
            node = Node(stop['stop_id'], stop['stop_name'], stop['stop_lat'], stop['stop_lon'])
            trips_from_node = connections[connections['src_id'] == stop['stop_id']]
            destinations_from_node = trips_from_node['dest_id'].unique()
            for destination in destinations_from_node:
                timetable = trips_from_node[trips_from_node['dest_id'] == destination].to_numpy()
                
                node.add_route_timetable(destination, timetable)
            self.nodes[stop['stop_id']] = node 
            connections_from_stop = walking_connections[walking_connections['source_id'] == stop['stop_id']]
            for _, connection in connections_from_stop.iterrows():
                node.add_walking_connection(connection.destination_id, connection.travel_time)
                

    def find_shortest_path(self, departing_stop_id, destination_stop_id, departure_timestamp, by_time, confidence = 0.9):
        """
        Finds shortest path with a time variant A* algorithm implementation 
        @param: departing_stop_id: stop id of the start position, string
        @param: destination_stop_id: stop id of the stop position, string
        @param: departure_timestamp: timestamp of trip initiation, string
        """
        # reset dynamic variables of all nodes
        for _, node in self.nodes.items():
            node.intialize_dynamic_parameters()
        
        # initialize departure node parameters
        departing_node = self.get_node(departing_stop_id)
        departing_node.is_under_consideration = True # halvveis redundant hotfix
        departing_node.elapsed_time = 0
        departure_time = pd.Timestamp(departure_timestamp)
        
        # initialize priority queue based on a function of the path distance and the heuristic distance
        nodes_under_consideration = [(0, departing_node)]
        heapq.heapify(nodes_under_consideration) 
        
        while len(nodes_under_consideration) > 0:
            (distance, current_node) = heapq.heappop(nodes_under_consideration) # unvisited node with lowest h(x) + g(x) 
            current_node.has_gained_optimum = True
            #print(f"evaluating stop {current_node.stop_id}")
            if current_node.stop_id == destination_stop_id:
                path = [] 
                while current_node.parent:
                    path.append({
                        "parent_stop_name":current_node.parent.stop_name,
                        "current_stop_name":current_node.stop_name,
                        "route":current_node.origin_route,
                        "successor_lat": current_node.lat,
                        "successor_lon": current_node.lon
                    })
                    current_node = current_node.parent
                return path[::-1] 
            
            # looping over all adjacent nodes
            successors_stop_ids = current_node.get_all_successor_stop_ids()
            for successor_stop_id in successors_stop_ids:
                successor_node = self.get_node(successor_stop_id)
                if successor_node.has_gained_optimum:
                    continue
                
                # evaluate alternative route to successor node through current node
                optimal_route = current_node.get_optimal_route(successor_node.stop_id, departure_time,
                                                               confidence, destination_stop_id, by_time)
                
                
                # trash hotfix of instance where there's no more routes for the day
                if not optimal_route or isinstance(optimal_route[1], float):
                    continue
                    
                # calculating the elapsed time to each adjacent node through the current node 
                # and updating the adjacent node's path if it's quicker
                (optimal_trip_proposal, optimal_elapsed_time_proposal) = optimal_route
                optimal_total_elapsed_time_proposal = current_node.elapsed_time + optimal_elapsed_time_proposal
                if successor_node.is_under_consideration:
                    if optimal_total_elapsed_time_proposal < successor_node.elapsed_time:
                        successor_node.elapsed_time = optimal_total_elapsed_time_proposal
                        successor_node.origin_route = optimal_trip_proposal
                        successor_node.parent = current_node

                else:
                    successor_node.elapsed_time = optimal_total_elapsed_time_proposal
                    successor_node.origin_route = optimal_trip_proposal
                    successor_node.parent = current_node
                    distance_heuristic = self.distance_heuristic(successor_node.stop_id, destination_stop_id)
                    heapq.heappush(nodes_under_consideration, (successor_node.elapsed_time + distance_heuristic, successor_node))
        return 'no_path'
    
    
    def get_node(self, stop_id):
        return self.nodes.get(stop_id)
    
    def distance_heuristic(self, stop_id, target_stop_id):
        """Finding the euclidean distance between a source stop and a target stop, This is used
        as a heuristic to speed up the A* algorithm"""
        node = self.get_node(stop_id)
        target_node = self.get_node(target_stop_id)
        alpha = 1
        distance = alpha*np.sqrt((node.lat - target_node.lat)**2 + (node.lon - target_node.lon)**2)
        return distance
    
    def get_connection_transfer_time(self, trip1, trip2):
        if trip1 is trip2:
            return 0
        return 2
    
    @staticmethod
    def pretty_print_route(path):
        def unit_to_text(quantity, unit):
            ret = str(quantity)+' '+ unit 
            if quantity != 1:
                ret += 's'
                
            return ret
        if path=='no_path':
            return False
        print(f"Trip No.{''.ljust(2)} Route{''.ljust(23)} Line{''.ljust(3)} Mode{''.ljust(4)} Departure time{''.ljust(7)} Departure stop {''.ljust(23)} Arrival time {''.ljust(8)} Arrival stop")
        print(f"{''.join(['-' for i in range(150)])}")
        for i, row in enumerate(path):
            if i>=10:
                break
            print(f"{str(i).ljust(10)} {row['route']['trip_id'].ljust(28)} {row['route']['route_short_name'].ljust(7)} {row['route']['route_desc'].ljust(7)}  {str(row['route']['src_departure_time']).ljust(21)} {row['parent_stop_name'].ljust(35)}->  {str(row['route']['dest_arrival_time']).ljust(21)} {row['current_stop_name']}")
        print(f"{''.join(['-' for i in range(150)])}")
        
        elapsed_time = pd.Timestamp(path[-1]['route']['dest_arrival_time']).to_pydatetime() - pd.Timestamp(path[0]['route']['src_departure_time']).to_pydatetime()
        hours, remainder = divmod(elapsed_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Total travel time: {unit_to_text(hours, 'hour')}, {unit_to_text(minutes, 'minute')} and {unit_to_text(seconds, 'second')}")

In [ ]:
%%local
# some helpers
def get_num_switch(sp):
    """Returns the number of switches for a shortest path"""
    count = 0
    for i, element in enumerate(sp):
        route = element.get('route')
        if i!=0:
            if route.get('route_short_name') != sp[i-1].get('route').get('route_short_name'):
                count+=1
    return count

In [ ]:
%%local    
# instantiate network
network = Network()
network.populate(stops, route_connections, walking_connections)

In [ ]:
%%local
source = '8591329'
destination = '8591220'
departure_time = '2022-05-26 10:24:00'
arrival_by = '2022-05-26 10:44:00' # this is redudant in this algorithm, but important for the final algorithm below
confidence = 0.7
# Saalsporthalle -> Kantonsschule, skal fungere helt fint siden de ligger på samme rute
%time sp = network.find_shortest_path(source, destination, departure_time, arrival_by, confidence)

The algorithm above, finds the fastest route from source to destination from a given time, but the query in the application is on arrival time. The query_routes method below takes care of this. The method works like a variant of binary search. We initialize by searching inside the hour before arrival by using binary search. For every route that meets the criteria, we save the route and continue the binary search until a route is found. This way, we return route options but guarantee to find the optimal one. If there is no route inside the hour, we search from 1 hour to 2 hours, then 2 hours to 4 hours, etc. This method gave a 5-fold speed increase compared to the naive implementation we first created which decremented time by one per search.

We realize that this algorithm is not the most effective for the routing. Instead, we could implement a reverse version of the A* where we start at arrival time from the destination and try to find departure node and departure time. This way, we would reduce the number of routing algorithms calls we do. We kept the A* propoagting forward as we initially thought we had to for the uncertainty modelling to work properly. Later, we understood that this was unnecessary as we would not use delay at previous stops as a predictior for delay at the current stop.

In [ ]:
%%local
import datetime

def query_routes(source='8591329', destination='8591220', by_time='2022-05-26 10:45:00', confidence=0.9):
    
    if (source not in stops.stop_id.values) or (destination not in stops.stop_id.values):
        print('Invalid stops')
        return False
    
    by_time = pd.Timestamp(by_time)
    routes = []
    confidence = float(confidence)
    
    # starting by looking for routes inside the hour
    max_time = 60
    min_time = 0

    # intitating the time difference
    delta_time = np.inf
    
    # keeping track  of where we have searched
    delta_times = []
    
    # running until optimal route found
    while True:
        
        # doubling the search space if we haven't found any routes
        if delta_time == max_time:
            max_time = max_time*2
        
        # how long back in time we search in this iteration
        delta_time = pd.Timedelta(minutes = int((max_time+min_time)/2))
        
        # if we already have used this delta_time, the search is completed.
        if delta_time in delta_times:
            break
        delta_times.append(delta_time)
        
        time_n = by_time - delta_time # the departure time to search from
        # finding shortest path
        sp = network.find_shortest_path(source, destination, str(time_n), str(by_time), confidence)
        # if no path, increase the time difference
        if sp=='no_path':
            min_time = delta_time.seconds//60
            continue
        arrival_time = pd.Timestamp(sp[-1].get('route').get('dest_arrival_time'))
        
        # if the path does not arrive fast enough, increasee time difference
        if arrival_time > by_time:
            min_time = delta_time.seconds//60 
            
        # if one finds a route, decrease time difference and see if we can find a faster route
        elif arrival_time <= by_time:
            max_time = delta_time.seconds//60
            if sp not in routes:
                routes.append(sp)
                
    return routes

In [ ]:
%%local
# testing the algorithm on a deafult route
%time routes = query_routes()

# some data manipulation for cleaner print
route_times = [route[0].get('route').get('src_arrival_time') for route in routes]
route_indices = np.flip(np.argsort(route_times))
routes = list(np.asarray(routes)[route_indices])
for i, sp in enumerate(routes):
    sp_start = sp[0].get('route').get('src_departure_time')
    sp_end = sp[-1].get('route').get('dest_arrival_time')
    print("%dth alternative:     Departure: %s.     Arrival: %s     Switches: %d"%(i+1, sp_start, sp_end, get_num_switch(sp)))

## Test of results

In this section, we will show the results of some tests, and provide an interface to play with the algorithm. But first, we define some functions for plotting.

In [ ]:
%%local
#create widgets to be used for visualization
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive, widgets, interact, Button, Text, HTML, VBox, HBox, Select, Layout
from IPython.display import display, clear_output


search_button = Button(description="Search Route")

source=Text(
        value='Zürich, Saalsporthalle',
        placeholder='Source destination',
        description = 'Origin:',
        disabled=False
    )

destination=Text(
    value='Zürich, Kantonsschule',
    placeholder='End destination',
    description='Destination:',
    disabled=False
)

time=Text(
    value='2022-05-26 10:45:00',
    placeholder='00:00',
    description='Arrival time:',
    disabled=False
)

certainty=Text(
    value='0.8',
    placeholder='0.8',
    description='Confidence level ([0,1)):',
    disabled=False,
    style = {'description_width': 'initial'}
)

arrow=HTML(
    value="<b>&rarr;</b>",
)

        
hcontainer = HBox(children=[source, arrow, destination])
hcontainer2 = HBox(children=[time, certainty])
vcontainer = VBox(children=[hcontainer, hcontainer2])

### Helping methods to display information
The methods below are used when ploting the trips found by our model.

In [ ]:
%%local
#Get stop_id from stop_name
def get_stop_id(stop_name):
    return stops[stops['stop_name']==stop_name].iloc[0]['stop_id']

#Transform a nested dictionary to a dataframe
def nested_dict_to_df(dic):
    return pd.json_normalize(dic, sep='_')
    
    
#Transform the route dictionary to the dataframe used in plotting the map
def route_to_correct_plot_format(route):
    #Add an extra row containing the first stop for easier plotting
    route = pd.concat([pd.DataFrame({'current_stop_name': route.iloc[0]['parent_stop_name'],
                                'route_cum_confidence':route.iloc[0]['route_cum_confidence'],
                                'successor_lat': network.get_node(route.iloc[0]['route_src_id']).lat,
                                'successor_lon': network.get_node(route.iloc[0]['route_src_id']).lon,
                                'route_trip_id': route.iloc[0]['route_trip_id'],
                                'route_dest_arrival_time': route.iloc[0]['route_src_arrival_time'],
                                'route_route_short_name': route.iloc[0]['route_route_short_name'],}, index=[0]),
                       route], ignore_index=True)
    route.rename(columns={'successor_lon': 'lon','successor_lat': 'lat'},inplace=True)
    
    #Add an column displaying if the stop is a stopover or not
    is_stop=list(map(int, route['route_route_short_name'].ne(route['route_route_short_name'].shift(periods=-1)).tolist()))
    is_stop[0]=1
    route['is_start_stop'] = is_stop
    
    return route


#Display the map on an easy readable format
def route_to_print(route):
    first = (route.groupby('route_route_short_name').apply(lambda x: x.iloc[0])
                   [['route_route_short_name', 'parent_stop_name', 'route_src_departure_time']]
                  )
    second = (route.groupby('route_route_short_name').apply(lambda x: x.iloc[-1])
                   [['current_stop_name', 'route_dest_arrival_time', 'route_cum_confidence']]
                  )
    
    finished = first.join(second)
    
    finished.columns = ['Route', 'Origin','Departure Time', 'Destination', 'Arrival Time', 'Confidence']
    finished = finished[['Route', 'Origin', 'Departure Time', 'Destination', 'Arrival Time', 'Confidence']]
    finished = finished.sort_values('Departure Time')
    return finished


#Create a list of what each marker should display when hovered over
def route_format_for_hovering(route, route_stopovers):
    route_info_origin=['{0}<br>Take route: {1} at {2}'.format(route['current_stop_name'].iloc[0],
                                                                         route['route_route_short_name'].iloc[0],
                                                                         route['route_dest_arrival_time'].iloc[0])]
    route_info=[]
    for i in range(1,route.shape[0]-1):
        if route['current_stop_name'].iloc[i] in (route_stopovers['Origin'].values):
            route_info.append('{0}<br>Arrive at {1}<br>Switch to route {2} leaving at {3}'
                          .format(route['current_stop_name'].iloc[i],
                                  route['route_dest_arrival_time'].iloc[i],
                                  route['route_route_short_name'].iloc[i+1],
                                  route['route_src_departure_time'].iloc[i+1]))
        else:
            route_info.append('')                      
    
    route_info_destination=['{0}<br>Arrive at {1}'.format(route['current_stop_name'].iloc[-1],
                                  route['route_dest_arrival_time'].iloc[-1])]
    return route_info_origin+route_info+route_info_destination

In [ ]:
%%local
#Set map token
mapbox_token = "pk.eyJ1IjoiYW5kcmVhc2FhcnIiLCJhIjoiY2wycndzcnhzMDBjdzNmcWxucGd5Y3U2ZiJ9.MVb6asnl_U4lPBj5KKv1DQ"
px.set_mapbox_access_token(mapbox_token)

#Plot the map
def display_map(route):
    #Transform the route dictionary to dataframe
    route=nested_dict_to_df(route)
    
    #Correct the format of the route for correct plotting
    route=route_to_correct_plot_format(route)
    
    #Plot map
    fig = go.Figure(go.Scattermapbox(
                    mode = "markers",
                    lat = route['lat'],
                    lon = route['lon'],
                    #Set dot settings
                    marker=go.scattermapbox.Marker(
                        color = 'green', 
                        size=route['is_start_stop']*10),
                    #Set hoverinfo to stop_name
                    text=route_format_for_hovering(route, route_to_print(route)),
                    hoverinfo='text'
        ))
    
    #Add lines to plot
    fig.add_trace(go.Scattermapbox(   
            lat = route['lat'],
            lon = route['lon'],
            mode = 'lines+text',
            line = dict(
                color = 'green',
            ),
            text=["Origin"]+["" for i in range(len(route)-2)]+["Destination"],
            textfont=dict(size=15),
            hoverinfo='none'
        )) 
    

    #Set initial map
    fig.update_layout(
        mapbox = {
            'accesstoken': mapbox_token,
            'style': "outdoors", 
            'zoom': 11,

            
            'center': go.layout.mapbox.Center(
                lat=47.3769,
                lon=8.5417
            )},
        width=1000,
        height=700,
        showlegend = False)
    
    fig.update_traces(visible=True, selector=dict(type='scattermapbox'))
    
    return fig

### Testing of code

### Test 1

In our first test we display a short trip within the Zurich city center. The trip goes from Zürich, Saalsporthalle to Zürich, Bahnhofquai/HB. 

In [ ]:
%%local
%time routes = query_routes('8591329', '8587349','2022-05-26 10:20:00.0', 0.8)
route_times = [route[0].get('route').get('src_arrival_time') for route in routes]
route_indices = np.flip(np.argsort(route_times))
routes = list(np.asarray(routes)[route_indices])

In [ ]:
%%local
route_to_print(nested_dict_to_df(routes[0]))

When comparing the alternative nr. 1 recommended by our model to the one on Google Maps, we get the exact same route and timetable. Below you see our route displayed

In [ ]:
%%local
display_map(routes[0])

The route Google Maps proposed:

https://drive.google.com/file/d/1QE0tMuJMgoFCP3LjJ_mgI-5613pNHvIG/view?usp=sharing

### Test 2

Here you see a more complex test. When comparing this route with the one proposed by Google, our model wants to depart at 14.15, while Google Maps proposes a route leaving at 14.27. This can be explained by one of the limits of our model, the maximum walking distance is set to 500 meters. While Google Maps suggests walking the last 600 meters, our model chooses to travel by bus one stop explaining why we need to leave early to make that bus.

In [ ]:
%%local
%time routes = query_routes('8591385', '8591053','2022-05-26 14:59:00.0', 0.6)
if len(routes)>0:
    route_times = [route[0].get('route').get('src_arrival_time') for route in routes]
    route_indices = np.flip(np.argsort(route_times))
    routes = list(np.asarray(routes)[route_indices])
else:
    print("We aplogize, no route meet these criteria")
    print("Please lower your confidence level or use a later arrival by time")

In [ ]:
%%local
route_to_print(nested_dict_to_df(routes[0]))

In [ ]:
%%local
display_map(routes[0])

The route Google Maps proposed:

https://drive.google.com/file/d/1BVGJceSUvgGFghel1VPSbSTB0eViH1As/view?usp=sharing

### Test 3

When we ran this test, we got a slightly better result than the route google maps proposed. This may be because route 916 no longer exists as Google Maps instead proposes walking to route 2.

In [ ]:
%%local
%time routes = query_routes('8591109', '8596007','2022-05-26 14:59:00', 0)
if len(routes)>0:
    route_times = [route[0].get('route').get('src_arrival_time') for route in routes]
    route_indices = np.flip(np.argsort(route_times))
    routes = list(np.asarray(routes)[route_indices])
else:
    print("We aplogize, no route meet these criteria")
    print("Please lower your confidence level or use a later arrival by time")

In [ ]:
%%local
route_to_print(nested_dict_to_df(routes[0]))

In [ ]:
%%local
display_map(routes[0])

The route Google Maps proposed:

https://drive.google.com/file/d/1h4BTUb1iMpqX433NAgRnju2wJyYDVTmz/view?usp=sharing


### Test 4
We did not find any route between the two locations in this test. This is contrary to our filtering which should ensure that the network is fully connected. We believe that an error in how we iterate over the walking connections can explain this. We haven't figured the exact problem, but we should at least change the function `get_all_successor_stop_ids` in the `Node` class. This is a limit of our model, which can be improved in future iterations.

In [ ]:
%%local
%time routes = query_routes('8572595', '8590565','2022-05-26 14:59:00.0', 0.6)
if len(routes)>0:
    route_times = [route[0].get('route').get('src_arrival_time') for route in routes]
    route_indices = np.flip(np.argsort(route_times))
    routes = list(np.asarray(routes)[route_indices])
else:
    print("We aplogize, no route meet these criteria")
    print("Please lower your confidence level or use a later arrival by time")

The route Google Maps proposes:
    
https://drive.google.com/file/d/1ipr41yssHcJU6Z2iEQ7vaunc8Tz5hi2C/view?usp=sharing

We did also test other routes, and the testing made us debug the code. For example, we realized we did not look up walking connections in the correct way.

# Application
By running the cell below, you can test our application. You type in an origin location, destination location, arrival time and confidence level and click the Search Route button. It may take some time to display the route, so you need to be patient.
When finished loading the different route alternatives, you get the option to choose which alternative you want to display. By clicking on one of them, you can see both the map showing the route and a table with the necessary information to tell you when to do switches. Also note the hovering information on the nodes. If you don't know what stops you want to search for, the very last cell of the notebook shows you some random stops you can use. If after some iterations, the plot does not work anymore, please run the cells below the title Test of Results, and it should work again.

In [ ]:
%%local
out = widgets.Output()

#Function run when search_button is clicked
@out.capture(clear_output=True)
def on_button_click(b):
    string='<b>Loading routes from {0} to {1}....</b>'.format(source.value, destination.value)
    display(HTML(string))
    try:
        #Extract the shortest path
        routes = query_routes(get_stop_id(source.value), get_stop_id(destination.value), time.value, float(certainty.value))
        w = widgets.Select(
        options=['{}th alternative'.format(i) for i in range(1,len(routes)+1)],
        #value='1th alternative',
        description='Select alternative to plot:',
        )

        output2 = widgets.Output()

        display(w, output2)
        def on_value_change(change):
            with output2:
                output2.clear_output()
                print("Chosen: " + change['new'])
                display(route_to_print(nested_dict_to_df(routes[int(change['new'][0])-1])))
                fig=display_map(routes[int(change['new'][0])-1])
                fig.show()

        w.observe(on_value_change, names='value')
    except IndexError:
        display(Text('Invalid stop name, try another stop'))
    
#Display the input fields
display(vcontainer)

#Load output when we click on the button
display(search_button)
display(out)
search_button.on_click(on_button_click)

In [ ]:
%%local
# run this cell if you want random suggestions to stops
stops.sample(frac=1).head(10)